In [14]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
from Utilities.symbolicFunctions import *
from Utilities.kukaKinematics import Slist, Mlist

# Task 3
## 3.2 
### Develop and implement a solution for the analytic inverse kinematics for the Agilus robot.

In [12]:
M = Mlist[5]
thetas_up = [0,0,0,0,0,0]
thetas_down = [0,0,0,0,0,0]

#Her endres thetas_gen for å teste forskjellige konfigurasjoner:
thetas_gen = np.array([2,1,4,2,1,2])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

P_s = PsFromTsd(T_sd)
print("Ps", P_s)

#theta1
thetas_up[0] = -sp.atan2(P_s[1],P_s[0]) #minus foran fordi vinkelen er definert andre vei
thetas_down[0] = thetas_up[0]

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
P_merket = np.array([P_s[0],P_s[1],P_s[2]-400])
b = np.sqrt((np.sqrt(P_merket[0]**2+P_merket[1]**2)-25)**2 + P_merket[2]**2)#Merk: -25 pga offset i skulder

psi = np.arccos(420/a) #Vinkelen mellom den faktiske armen og den vi tegna for å få en trekant(Pga 35mm offset i elbow). Se notatbok
phi = sp.atan2(P_merket[2], sp.sqrt(P_merket[0]**2 + P_merket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c)) #Vil ha negativ alpha når b krysser offesettet, som i null-posisjonen
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

print("alpha:", alpha, "beta:", beta, "phi",phi, psi, "b:", b)

#thetas_up[2] = np.pi/2 - (beta-psi)
#thetas_down[2] =  -(2*np.pi - (beta+psi) - np.pi/2)
thetas_down[1] =  -(phi-alpha)
thetas_up[2] = np.pi - beta + psi
if(thetas_up[2]<0):
    alpha = -alpha
#thetas_down[2] =  -thetas_up[2]
thetas_down[2] =  -(np.pi - beta - psi)

#thetas_up[1] = np.pi/2 - (alpha + phi) #Index 1 på thetas[1] og thetas[2] gir elbow down løsning
thetas_up[1] = -(alpha + phi)
#thetas_down[1] =  np.pi/2 - (phi-alpha)


print(thetas_up, thetas_down)

#Vi har XYX euler angles. De er egentlig (-X)Y(-X) fordi det er slik S'ene er definert,.

#Elbow down:
T1 = exp6(Slist[:,0], -thetas_down[0])
T2 = exp6(Slist[:,1], -thetas_down[1])
T3 = exp6(Slist[:,2], -thetas_down[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M)) #R er den resterende rotasjonen vi ønsker å få fra de tre siste leddene, definert i s

thetas_down[3] = -sp.atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetas_down[4] = sp.atan2(sp.sqrt(1-R[0,0]**2), R[0,0])
thetas_down[5] = -sp.atan2(R[0,1], R[0,2])


#Elbow up:
T1 = exp6(Slist[:,0], -thetas_up[0])
T2 = exp6(Slist[:,1], -thetas_up[1])
T3 = exp6(Slist[:,2], -thetas_up[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M))

thetas_up[3] = -sp.atan2(R[1,0], -R[2,0])
thetas_up[4] = sp.atan2(sp.sqrt(1-R[0,0]**2), R[0,0])
thetas_up[5] = -sp.atan2(R[0,1], R[0,2])

T_sd
 [[-7.8126e-01  7.4168e-02  6.1979e-01 -1.9790e+02]
 [-3.0648e-01  8.1941e-01 -4.8439e-01 -2.8533e+02]
 [-5.4379e-01 -5.6839e-01 -6.1744e-01  4.7920e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
Ps [-148.3203 -324.0857  429.8071    1.    ]
alpha: 1.0896979375776463 beta: 0.7752661145217655 phi 0.0896979375776459 0.08314123188844062 b: 332.75103099820745
[-1.14159265358979 + pi, -1.17939587515529, 2.4494677709564683, -1.55961402768068 + pi, -0.871344541900283 + pi, -0.421451417169787] [-1.14159265358979 + pi, 1.00000000000000, -2.2831853071795867, -1.14159265358979 + pi, 1.00000000000000, -1.14159265358979 + pi]


## Task 3.3
### Confirm that the solution of the analytical inverse kinematics from the previous point agrees with the solution from a numerical inverse kinematics solver.


In [15]:
#UP
thetas_up_num = np.zeros(6)
thetas_up_deg = np.zeros(6)
for i in range(0,6):
    thetas_up_num[i] = sp.N(thetas_up[i])
    thetas_up_deg[i] = np.rad2deg(thetas_up_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_up_thetas = mr.FKinSpace(M,Slist,thetas_up_num)
P_up_reached = PsFromTsd(T_up_thetas)
#print(P_reached, P_s)
np.set_printoptions(precision=4)

#DOWN
thetas_down_num = np.zeros(6)
thetas_down_deg = np.zeros(6)
for i in range(0,6):
    thetas_down_num[i] = sp.N(thetas_down[i])
    thetas_down_deg[i] = np.rad2deg(thetas_down_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_down_thetas = mr.FKinSpace(M,Slist,thetas_down_num)
P_down_reached = PsFromTsd(T_down_thetas)


#fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
#R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 
thetas_calc, asd = mr.IKinSpace(Slist,M,T_sd,[0,0,0,0,0,0],0.01,0.01)

T_sd, thetas_gen, T_up_thetas, thetas_up_num, T_down_thetas, thetas_down_num, P_s, P_up_reached, P_down_reached

(array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([2, 1, 4, 2, 1, 2]),
 array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([ 2.    , -1.1794,  2.4495,  1.582 ,  2.2702, -0.4215]),
 array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([ 2.    ,  1.    , -2.2832,  2.    ,  1.    ,  2.    ]),
 array([-148.3203, -324.0857,  429.8071,    1.    ]),
 array([-148.3203, -324.0857,  429.8071,

In [7]:
T_sd, thetas_up_num, thetas_down_num

jointLimits = np.array([[-180, 180], [-190, 45], [-120, 156], [-180, 180], [-90, 90], [-180, 180]]) #Assuming joint 5 has limits [-90, 90]

def applyJointLim(jointLimits, thetas):
    ''' Check if Inverse Kinematics solution (thetas) is within jointlimits\n 
    PARAMETERS:
    Jointlimits, numpy 2D array with lower and upper limits in deg\n
    Jointangles, numpy 1D array in rad
    RETURNS: Boolean true or false, if false a print message with the offending link is printed '''
    jointLimits = np.deg2rad(jointLimits) 
    thetas %= 2*np.pi #Post processing: all thetas in [0,2*pi)

    for theta, i in enumerate(thetas):
        if jointLimits[i][1] < theta < jointLimits[i][0]:
            print("Joint number: ", i+1, "is not within the limits")
            return False
    return True


## Task 3.4
### Using the developed analytic inverse kinematics formulation, visualize the Agilus robot in both elbow-up and elbow-down configurations for the same end-effector pose.

In [36]:
from Utilities.RobotClass import * 

In [37]:
Kuka = Robot(Mlist, ['z', '-z', 'x', 'x', '-z','x']) #Initialized the kuka-robot object in zero-configuration
Kuka.draw_robot()

WebVisualizer(window_uid='window_23')

In [38]:
Kuka.transform(Slist, thetas_down_num ) # Elbow DOWN
pUp = np.array(Kuka.joints[5].coord.get_center()) #Get coordinates of {6} in elbow down 
Kuka.draw_robot()

Kuka.transform(Slist, thetas_up_num ) # Elbow UP
pDown = np.array(Kuka.joints[5].coord.get_center(),dtype=float)  #Get coordinates of {6} in elbow up
Kuka.draw_robot()

print(pUp, pDown)
print("Both elbow up and elbow down config yields the same end-effector pose: ",pUp.round(4) == pDown.round(4))

WebVisualizer(window_uid='window_24')

WebVisualizer(window_uid='window_25')

[-198.0161 -285.2979  476.9679] [-198.0161 -285.2979  476.9679]
Both elbow up and elbow down config yields the same end-effector pose:  [ True  True  True]
